In [1]:
#!unzip tensortrade.zip
#!unrar x LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022.rar

In [2]:
##get data from link
#!wget -O data.rar https://www.dropbox.com/sh/67003kc9fekby1h/AAACExVV_b0hGQ8SdclrAX6ua/LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022.rar?dl=0
#!unrar x '/content/data.rar'

In [1]:
## test if using gpu or not # tensorflow
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')

In [26]:
# test and use gpu # pytorch
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [1]:
# !pip install tabulate
# !pip install stable_baselines3
# !pip install gym
# !pip install deprecated
# !pip install stockstats
# !pip install zigzag
# !pip install unrar
# !pip install optuna

In [2]:
##################### Base Libs #####################
#from tensortrade.data.inputs import *
from tensortrade.data.feature_engineering import FeatureEngineering
import tensortrade.env.default as default
import tensortrade.env.env_stocktrading_train as Environment_Train
import tensortrade.env.env_stocktrading_test as Environment_Test
################### Others ##########################
import numpy as np
import pandas as pd
import json
import optuna
import time

In [3]:
Config_Path= "E:/Tensurf-RL/tensurfrl/TensorTrade/Tensor-0.1/configuration.json"
Config_File= open(Config_Path)
Config = json.load(Config_File)

In [4]:
'''


Data Ghathering 



'''

'\n\n\nData Ghathering and Engineering\n\n\n\n'

In [5]:
# see data path
print('Data Path is :',Config['Data_Path'])

Data Path is : C:/Users/kasra/Downloads/Ten-Surf/Tensurf-RL/data/LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022.csv


In [6]:
# Primary feature engineering

import shutil
import ntpath
import os
import numpy as np
import math
####################################

def k_moving_average(arr, window_size=20):
    moving_averages = []
    i = 0
    while i < len(arr) - window_size + 1:
        window_average = round(np.sum(arr[i:i+window_size]) / window_size, 2)
        moving_averages.append(window_average)
        i += 1
    return moving_averages
    
def process_data(path, ind, n, first_line, cols, dirname, level=10, k=[5,10,20,50,100], look_back=50):
    data = [first_line]
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i >= ind[0] and i < ind[1]:
                data.append(line)
    ltps = []
    for line in data[1:]:
        splits = line.split('|')
       
        ltp = float(splits[cols['Candle_LastTradePrice']])
        ltps.append(ltp)
    multi_labels = []
    for h in k:
        ma = k_moving_average(ltps, h)
        k_minus = np.array([np.nan] * (h-1) + ma)
        k_plus = np.array(ma + [np.nan] * (h-1))
        smoothing = (k_plus - k_minus) / k_minus
        alpha = np.std(smoothing[~np.isnan(smoothing)]) / 2
        labels = []
        for s in smoothing[~np.isnan(smoothing)]:
            if s > alpha:
                labels.append(s)
            elif s < -alpha:
                labels.append(s)
            else:
                labels.append(s)
        labels_str = [np.nan] * (h-1) + labels + [np.nan] * (h-1)
        labels_str = [str(i) for i in labels_str]
        multi_labels.append(labels_str)
    data[0] = data[0][:-1] + '|Label1|Label2|Label3|Label4|Label5\n'
    for i in range(len(data)):
        if i == 0:
          continue
        else:
          splits = data[i].split('|')
          bidprices = splits[cols['LOB_BidPrices']].split(',')
          bidprices = [p for p in bidprices if p != ''][-level:]
          askprices = splits[cols['LOB_AskPrices']].split(',')
          askprices = [p for p in askprices if p != ''][:level]
          bidvolumes = splits[cols['LOB_BidVolumes']].split(',')
          bidvolumes = [v for v in bidvolumes if v != ''][-level:]
          askvolumes = splits[cols['LOB_AskVolumes']].split(',')
          askvolumes = [v for v in askvolumes if v != ''][:level]
          splits[cols['LOB_BidPrices']] = ','.join(bidprices)
          splits[cols['LOB_AskPrices']] = ','.join(askprices)
          splits[cols['LOB_BidVolumes']] = ','.join(bidvolumes)
          splits[cols['LOB_AskVolumes']] = ','.join(askvolumes)
          splits[-1] = splits[-1][:-1]
          splits.append(multi_labels[0][i-1])
          splits.append(multi_labels[1][i-1])
          splits.append(multi_labels[2][i-1])
          splits.append(multi_labels[3][i-1])
          splits.append(multi_labels[4][i-1] + '\n')
          data[i] = '|'.join(splits)
    filename = ntpath.basename(path).split('.')[0] + 'labeled{}.csv'.format(n)
    with open(os.path.join(dirname, 'labeled', filename), 'w') as f:
        for line in data:
            f.write(line)
    corrupts = []
    for i, line in enumerate(data):
        splits = line.split('|')
        bidprices = splits[cols['LOB_BidPrices']].split(',')
        askprices = splits[cols['LOB_AskPrices']].split(',')
        bidvolumes = splits[cols['LOB_BidVolumes']].split(',')
        askvolumes = splits[cols['LOB_AskVolumes']].split(',')
        if len(bidprices) < level or len(askprices) < level or '' in askprices or '' in bidprices:    # corrupt condition
            corrupts.append(i)
    print(len(corrupts))
    subfiles = []
    for i in range(len(corrupts) - 1):
        start = corrupts[i]
        end = corrupts[i+1]
        if end - start - 1 >= look_back:
            subfiles.append(data[start+1:end])
    if (len(data) - 1 - corrupts[-1]) >= look_back:
        subfiles.append(data[corrupts[-1]+1:])
    total = []
    for f in subfiles:
        total += f
    filename = ntpath.basename(path).split('.')[0] + '_clean_labeled_{}.csv'.format(n)
    with open(os.path.join(dirname, 'clean_labeled', filename), 'w') as f:
        for line in total:
            f.write(line)
    samplesX = []
    samplesY = []
    for f in subfiles:
        for i in range(look_back-1, len(f)):
            splits = f[i].split('|')
            sample_labels = [float(splits[-5])+1, float(splits[-4])+1, float(splits[-3])+1, float(splits[-2])+1, float(splits[-1][:-1])+1]
            if any([math.isnan(i) for i in sample_labels]):
                continue
            total_ask_prices = []
            total_ask_volumes = []
            total_bid_prices = []
            total_bid_volumes = []
            for j in range(i+1-look_back, i+1):
                spltis = f[j].split('|')
                total_ask_prices = total_ask_prices + splits[cols['LOB_AskPrices']].split(',')
                total_bid_prices = total_bid_prices + splits[cols['LOB_BidPrices']].split(',')
                if '' in total_bid_prices:
                    print(splits[cols['LOB_BidPrices']])
                    return splits[cols['LOB_BidPrices']]
                total_ask_volumes = total_ask_volumes + splits[cols['LOB_AskVolumes']].split(',')
                total_bid_volumes = total_bid_volumes + splits[cols['LOB_BidVolumes']].split(',')
            total_ask_prices = np.array([float(i) for i in total_ask_prices])
            total_ask_volumes = np.array([float(i) for i in total_ask_volumes])
            total_bid_prices = np.array([float(i) for i in total_bid_prices])
            total_bid_volumes = np.array([float(i) for i in total_bid_volumes])
            price_mean = np.concatenate([total_ask_prices, total_bid_prices]).mean()
            price_std = np.concatenate([total_ask_prices, total_bid_prices]).std()
            volume_mean = np.concatenate([total_ask_volumes, total_bid_volumes]).mean()
            volume_std = np.concatenate([total_ask_volumes, total_bid_volumes]).std()
            total_ask_prices = (total_ask_prices - price_mean) / price_std
            total_ask_volumes = (total_ask_volumes - volume_mean) / volume_std
            total_bid_prices = (total_bid_prices - price_mean) / price_std
            total_bid_volumes = (total_bid_volumes - volume_mean) / volume_std
            total = np.stack([total_ask_prices, total_bid_prices, total_ask_volumes, total_bid_volumes]).T
            total = total.reshape((look_back, -1))
            samplesX.append(total)
           
            samplesY.append(sample_labels)
    X_name = ntpath.basename(path).split('.')[0] + 'X{}.npy'.format(n)
    y_name = ntpath.basename(path).split('.')[0] + 'y{}.npy'.format(n)
    with open(os.path.join(dirname, 'numpy', X_name), 'wb') as f:
        np.save(f, np.array(samplesX).reshape(-1, look_back, level*4, 1))
    with open(os.path.join(dirname, 'numpy', y_name), 'wb') as f:
        np.save(f, np.array(samplesY))

def prepare_data_2(path, level=10, k=[5,10,20,50,100], look_back=50):
    dirname = ntpath.basename(path).split('.')[0] + '_processed'
    count = 0
    with open(path, 'r') as f:
        for line in f:
            if count == 0:
                cols = line.split('|')
                cols[-1] = cols[-1][:-1]
                cols = {cols[i]:i for i in range(len(cols))}
                first_line = line
            count += 1
   
    n = (count // 45000) + 1
    
    inds = []
    for i in range(n):
        if (count - i*45000) > 45000:
            if i == 0:
                inds.append((1, (i+1)*45000))
            else:
                inds.append((i*45000, (i+1)*45000))
        else:
            inds.append((i*45000, count))
    try:
        shutil.rmtree(dirname)
        print('Removing old directory and creating a new one')
    except:
        print('No such directory')
        print('Creating new directory')
        
    os.mkdir(dirname)
    os.mkdir(os.path.join(dirname, 'labeled'))
    os.mkdir(os.path.join(dirname, 'clean_labeled'))
    os.mkdir(os.path.join(dirname, 'numpy'))

    for i, ind in enumerate(inds):
        process_data(path=path, ind=ind, n=i, first_line=first_line, cols=cols, dirname=dirname, level=level, k=k, look_back=look_back)

#######################################################

In [ ]:
prepare_data_2(Config['Data_Path'])

Removing old directory and creating a new one
56


C:\Users\kasra\AppData\Local\Temp\ipykernel_5968\2734505880.py:133: RuntimeWarning: invalid value encountered in divide
  total_ask_volumes = (total_ask_volumes - volume_mean) / volume_std
C:\Users\kasra\AppData\Local\Temp\ipykernel_5968\2734505880.py:135: RuntimeWarning: invalid value encountered in divide
  total_bid_volumes = (total_bid_volumes - volume_mean) / volume_std


1618
1


c:\Users\kasra\.conda\envs\Ai\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\kasra\.conda\envs\Ai\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\kasra\.conda\envs\Ai\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
column_name= ['DateTime','open', 'high', 'low',
       'close', 'Candle_LastTradePrice', 'volume',
       'bid_volume', 'ask_volume',
       'numTrades', 'bid_number',
       'ask_number', 'askbidDiffHigh',
       'askbidDifflow', 'askbidnumTradesDiffHigh',
       'askbidnumTradesDifflow', 'UpDownvolDiffHigh',
       'UpDownvolDifflow', 'ATR', 'sum_lob_bid', 'sum_lob_ask',
       'LOB_SumBidTick', 'LOB_SumAskTick', 'lob_bid_price', 'lob_bid_volume',
       'lob_ask_price', 'lob_ask_volume', 'VAP_Prices', 'VAP_Volumes',
       'VAP_AskVolumes', 'VAP_BidVolumes', 'VAP_NumberOfTrades',
       'VAP_TotalVolume']

In [8]:
D1=pd.read_csv('LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_processed/clean_labeled/LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_clean_labeled_0.csv',names= column_name, usecols = [i for i in range(33)], delimiter="|")
D2=pd.read_csv('LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_processed/clean_labeled/LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_clean_labeled_1.csv',names= column_name, usecols = [i for i in range(33)], delimiter="|")
D3=pd.read_csv('LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_processed/clean_labeled/LOB1_NQU22-CME_1min_2PercentSum_100PercentOrders_Overlapped_20Jun2022_19Sep2022_clean_labeled_2.csv',names= column_name, usecols = [i for i in range(33)], delimiter="|")

In [9]:
Data= pd.concat([D1,D2,D3])
Data["DateTime"] = pd.to_datetime(Data["DateTime"])
Data.set_index(["DateTime"], inplace=True)

In [10]:
Data.head(3)

,open,high,low,close,Candle_LastTradePrice,volume,bid_volume,ask_volume,numTrades,bid_number,...,lob_bid_price,lob_bid_volume,lob_ask_price,lob_ask_volume,VAP_Prices,VAP_Volumes,VAP_AskVolumes,VAP_BidVolumes,VAP_NumberOfTrades,VAP_TotalVolume
DateTime,,,,,,,,,,,,,,,,,,,,,
2022-06-19 23:59:00,11385.5,11385.50,11378.00,11379.25,11378.50,129,66,63,123,66,...,"11383.00,11383.25,11383.50,11383.75,11384.00,1...","1,2,1,1,1,2,2,1,1,1","11378.25,11378.50,11378.75,11379.00,11379.25,1...","2,1,1,1,2,1,2,2,2,1","11378.00,11378.25,11378.50,11378.75,11379.00,1...","3,3,12,32,15,17,14,5,8,1,2,2,1,2,1,2,1,4,1,1,1,1,","0,0,3,20,9,11,8,2,7,1,0,0,0,0,0,0,0,2,0,0,0,0,","3,3,9,12,6,6,6,3,1,0,2,2,1,2,1,2,1,2,1,1,1,1,","3,3,12,29,14,17,14,5,6,1,2,2,1,2,1,2,1,4,1,1,1,1,",129
2022-06-20 00:00:00,11378.5,11387.25,11369.50,11381.25,11382.00,343,171,172,327,170,...,"11384.50,11384.75,11385.00,11385.25,11385.50,1...","2,1,1,1,1,2,2,3,2,1","11370.00,11370.25,11370.50,11370.75,11371.00,1...","1,1,2,2,2,1,1,2,1,1","11369.50,11369.75,11370.00,11370.25,11370.50,1...","1,1,4,1,1,5,1,1,2,1,10,6,7,3,13,2,2,3,2,2,3,2,...","0,0,1,1,1,1,0,0,1,1,0,3,3,0,10,1,1,2,2,1,1,2,2...","1,1,3,0,0,4,1,1,1,0,10,3,4,3,3,1,1,1,0,1,2,0,2...","1,1,4,1,1,5,1,1,2,1,10,6,7,3,10,2,2,3,2,2,3,2,...",343
2022-06-20 00:01:00,11382.0,11397.25,11379.75,11395.50,11397.25,282,84,198,255,83,...,"11394.25,11394.50,11394.75,11395.00,11395.25,1...","4,4,3,1,1,3,4,3,1,2","11380.25,11380.50,11380.75,11381.00,11381.25,1...","1,1,14,1,1,1,1,1,1,1","11379.75,11380.25,11380.50,11380.75,11381.00,1...","1,4,3,26,2,4,1,2,7,3,3,2,3,4,3,7,3,4,9,4,3,2,2...","0,0,1,25,0,4,0,0,4,0,2,1,2,0,0,5,2,0,9,2,2,1,1...","1,4,2,1,2,0,1,2,3,3,1,1,1,4,3,2,1,4,0,2,1,1,1,...","1,4,3,8,2,4,1,2,7,3,3,2,3,3,3,7,3,4,9,4,3,2,2,...",282


In [1]:
## LOB Data
# LOB_Data_test_unnormal.head(3)

In [13]:
'''


Train Test and Optimize



'''

'\n\n\nTrain Test and Optimize\n\n\n\n'

In [14]:
# func to create environment

def create_env_train(data,price,date,config):

    env= Environment_Train.StockTradingEnv_Train(data,price,date,config)
    
    return env

In [15]:
# func to create test environment

def create_env_test(data,price,date,config):

    env= Environment_Test.StockTradingEnv_Test(data,price,date,config)

    return env

In [18]:
# import agents
from stable_baselines3 import A2C,DQN,PPO
# import environment checker
from stable_baselines3.common import env_checker

In [19]:
# # check if environment meet gym policy
# env_checker.check_env(env_train , warn=False, skip_render_check=True)

In [20]:
# import torch.nn.functional as F
# from torch.utils import data
# from torchinfo import summary
# from typing import Callable, Dict, List, Optional, Tuple, Type, Union
# from stable_baselines3.common.policies import ActorCriticPolicy

In [21]:
import gym
import torch
import torch as th
import torch.nn as nn
#from torch import nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int ):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.shape[0]
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,4), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,4), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
             nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        # Compute shape by doing one forward pass
        # with th.no_grad():
        #     n_flatten = self.inp3(
        #         th.as_tensor(observation_space.sample()[None]).float()
        #     ).shape[1]
        
        # lstm layers
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, features_dim)

    def forward(self, x: th.Tensor) -> th.Tensor:
        
        # return self.linear(self.lstm(self.cnn(observations)))
        # h0: (number of hidden layers, batch size, hidden size)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)
       
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)  
        
        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
#         x = torch.transpose(x, 1, 2)
        x = x.permute(0, 2, 1, 3)
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        # forecast_y = torch.softmax(x, dim=1)
        return x

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=49),
    net_arch=[dict(vf=[64], pi=[64])]
)

In [27]:
def Main(config, trial=None):
    
    if config['RUNNING_MODE'] == 'optimize':
        config = select_params_value(trial, config)

    # Data Feature Engineering
    preProcessedData = FeatureEngineering(Data,Config['Feature_engineering'])
    LOB_Data_train,LOB_Data_test,LOB_Data_train_unnormal,LOB_Data_test_unnormal,LOB_Train_dates,LOB_Test_dates = preProcessedData.add_all_features()

    # make train data
    Time_Data = LOB_Train_dates # just time column
    Data=LOB_Data_train
    Price_Data = pd.DataFrame(LOB_Data_train_unnormal[['open','high','low','close']]) # just price related columns

    # make test data
    Data_Test=LOB_Data_test
    Price_Data_Test = pd.DataFrame(LOB_Data_test_unnormal[['open','high','low','close']]) # just price related columns
    #Price_Data_Test.reset_index(drop=True,inplace=True)
    Time_Data_Test = LOB_Test_dates  # just time column

    # Number of objectives
    obj_no = config['number_of_objectives']

    # create train env
    env_train= create_env_train(Data.copy(),Price_Data.copy(),Time_Data,config['env_config_train'])

    # create test env
    env_test =  create_env_test(Data_Test.copy(),Price_Data_Test.copy(),Time_Data_Test,config['env_config_test'])

    # create RL model
    #policy_kwargs = dict(net_arch=[8,16, dict(vf=[32,64,128,256], pi=[32,64,128,256])])
    RL_model = A2C(config['A2C_PARAMS']['net_arch'], env_train, verbose=1,device=device,
    learning_rate=config['A2C_PARAMS']['learning_rate'], policy_kwargs=policy_kwargs,seed=config['A2C_PARAMS']['SEED'])
    RL_model.learn(total_timesteps=config['A2C_PARAMS']['TOTAL_TIMESTEPS'])
    #RL_model.save("RlSurf-A2C")

    # test RL model on test data
    obs = env_test.get_state()    #~ we should not reset env manullay
    number_candle=0 

    while True: 
    
        action, _states = RL_model.predict(obs,deterministic=True)
    
        number_candle+=1
        obs, rewards, done, info = env_test.step(action)

        if done:

            if env_test.day_index>= len(env_test.day_indices)-1:
                    print("Account Balance Is : ", info['account_status'])
                    
                    break

            else:
                print('episode(Day) is : ',env_test.episode)
                env_test.reset()

    if config['RUNNING_MODE'] in ('backtest','train') :
            print(env_test.tradeslist)
            env_test.show_results()

    elif Config['RUNNING_MODE'] == 'optimize':
            cum_pnl,maxdrawdown= env_test.show_results()
            if obj_no == 1:
                return cum_pnl
            elif obj_no == 2:
                return cum_pnl, maxdrawdown

############## change config for each trial               
def select_params_value(trial, config):
    
    optimization_params = json.load(open('E:/Tensurf-RL/tensurfrl/TensorTrade/Tensor-0.1/optimization_params.json'))
    for key, value in optimization_params.items():
        if key.startswith('_'):
            continue
      
        param_value = getattr(trial, value[0])(key, **value[1])

        if key in config['A2C_PARAMS'].keys():
            config['A2C_PARAMS'][key] = param_value

        elif key in config['env_config_train'].keys():
            config['env_config_train'][key] = param_value

        elif key in config['env_config_test'].keys():
            config['env_config_test'][key] = param_value
        
        elif key in config['Feature_engineering'].keys():
            config['Feature_engineering'][key] = param_value

        else:
            raise Exception('Undefined parameter ' + key)
            
    return config

In [28]:
# start time
start = time.time()

# check run mode and make study with optuna
if Config['RUNNING_MODE'] == 'optimize':
    
    study_name = Config['study_name']
    storage = 'sqlite:///optimization_results.sqlite'
    try:
        obj_no = Config['number_of_objectives']
        directions = ["maximize","minimize"] 

        if obj_no == 1:
            study = optuna.create_study(direction="maximize", study_name=study_name, storage=storage)
        else:
            study = optuna.create_study(directions=directions , study_name=study_name, storage=storage)
    except:
        study = optuna.load_study(study_name=study_name, storage=storage)
        print('- - - - - - Optimization study loaded - - - - - -')
    study.optimize(
        lambda trial: Main(Config, trial), n_trials=100, n_jobs=1, timeout=None)
        
else:
      Main(Config)

# total elapsed time
print(f'Runtime: {round(time.time() - start)} secs')

[I 2022-10-26 10:56:00,715] A new study created in RDB with name: optimization1
c:\Users\kasra\.conda\envs\Ai\lib\site-packages\optuna\distributions.py:683: UserWarning: The distribution is specified by [1, 20] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 19].
  warnings.warn(
c:\Users\kasra\.conda\envs\Ai\lib\site-packages\optuna\distributions.py:683: UserWarning: The distribution is specified by [1, 10] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 9].
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 38       |
|    iterations         | 100      |
|    time_elapsed       | 13       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.842   |
|    explained_variance | 0.366    |
|    learning_rate      | 0.001    |
|    n_updates          | 99       |
|    policy_loss        | 0.961    |
|    value_loss         | 1.27     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 39       |
|    iterations         | 200      |
|    time_elapsed       | 25       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.765   |
|    explained_variance | 0.152    |
|    learning_rate      | 0.001    |
|    n_updates    

[W 2022-10-26 10:57:01,897] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\Users\kasra\.conda\envs\Ai\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\kasra\AppData\Local\Temp\ipykernel_14816\3828216677.py", line 19, in <lambda>
    lambda trial: Main(Config, trial), n_trials=100, n_jobs=1, timeout=None)
  File "C:\Users\kasra\AppData\Local\Temp\ipykernel_14816\3108804324.py", line 17, in Main
    RL_model.learn(total_timesteps=config['A2C_PARAMS']['TOTAL_TIMESTEPS'])
  File "c:\Users\kasra\.conda\envs\Ai\lib\site-packages\stable_baselines3\a2c\a2c.py", line 197, in learn
    return super().learn(
  File "c:\Users\kasra\.conda\envs\Ai\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py", line 267, in learn
    self.train()
  File "c:\Users\kasra\.conda\envs\Ai\lib\site-packages\stable_baselines3\a2c\a2c.py", line 170, in train
    th.nn

KeyboardInterrupt: 

In [56]:
## Alt Env Test

# ################ Libs ##############
# import numpy as np
# import pandas as pd
# from gym.utils import seeding
# import gym
# from gym import spaces
# import matplotlib
# import matplotlib.pyplot as plt
# from stable_baselines3.common.vec_env import DummyVecEnv
# from stockstats import StockDataFrame 
# from zigzag import peak_valley_pivots
# from tabulate import tabulate
# from scipy.signal import argrelextrema
# ################################

# """
# A custom stock trading environment based on OpenAI gym (Test Env)

# """

# class ENVV(gym.Env):

#     def __init__(self,df, price_data, date, config):

#         self.data = df
#         self.config = config
#         self.price_data = price_data
#         self.price_data['action'] = 0   # make action column in price data to save each action
#         self.atr = StockDataFrame.retype(price_data.copy())['atr'] # calculate atr indicator
#         self.timesteps = self.config["TIMESTEPS"] 
#         self.action_space = spaces.Discrete(3) # define action space
#         self.state_space = (self.action_space.n * config["USE_LAST_ACTION"]) + 2 * config[ 
#             "USE_CURRENT_PNL"] + self.timesteps * len(df.columns)  # define states space
        
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(1,100,49)) # don't change this line !!!
#         self.num_of_shares = config["MAX_NUM_SHARES"]
#         self.date = date # date of dataframe

#         daily = pd.to_datetime([pd.to_datetime(x) - pd.Timedelta(hours=15) for x in self.date]).floor('d') # specify day indexes
#         daily = pd.DataFrame(daily, columns=['date'])
#         self.day_indices = [x.total_seconds() for x in daily['date'] - daily['date'].shift(1)]
#         self.day_indices = [i for i, v in enumerate(self.day_indices) if v != 0]
#         self.day_indices = self.day_indices[1:] 
#         self.day_indices[0] = self.timesteps 

#         self.current_point = self.day_indices[0] 
#         self.day_index = 0 
#         self.current_position = 0
#         self.last_action = 0
#         self.entry_index = 0
       
#         self.spread_cost = config["Exchange_Commission"]    # set exchange commission
        
#         self.base_account = config["INITIAL_ACCOUNT_BALANCE"]

#         self.episode = 0
#         self.account = self.base_account
#         # memorize all the total balance change
#         self.entry_date = None
#         self.tradeslist = pd.DataFrame( 
#             columns=['entry date', 'entry price', 'action', 'exit date', 'exit price', 'pnl','cum_pnl', 'mae', 'mpe','exit mode','sl','tp'])
#         self.tradeslist.loc[0] = [self.date[0],0, 0,
#         self.date[0], 0,0, self.account, 0, 0, 0, 0, 0]
#         self._seed()
#         self.positives = 0    # ?
#         self.negatives = 0    # ?

#         self.sl_mode= config["SL_mode"]
#         self.sl_value= config["STOP_LOSS"]                                
#         self.scipy_neighborhood=  config["Scipy_neighborhood"]
#         self.R2R = config["R2R"]
#         self.stop_loss= 0
#         self.take_profit = -1
#         self.sltp_flag= False

#     '''
#     func to determine stop loss base on zigzag indicator

#     '''
#     def find_pivots(self, action):

#         pivot_param = .002
#         step = .0005
#         lookback = self.config['lookback']
#         data = self.price_data['close'][max(0, self.current_point - lookback):self.current_point].to_numpy()
#         pivots = peak_valley_pivots(data, pivot_param, -pivot_param)
#         condition = -action
#         current_price = self.price_data['close'][self.current_point - 1]
#         if np.sum(pivots == condition) and condition * current_price < condition * data[
#             np.where(pivots == condition)[0][-1]]:
#             return abs(current_price - data[np.where(pivots == condition)[0][-1]])
#         else:
#             return self.config["STOP_LOSS_COEF"] * self.atr[self.current_point]     

#     '''
#     func to get date

#     '''
#     def get_date(self):
        
#         try:
#             return self.date[self.current_point]
#         except:
#             pass

#     '''
#     func to get price

#     '''
#     def get_price(self, point):

#         price_type = self.config["PRICE_CALCULATION_TYPE"]

#         if price_type == 'mid':
#             return self.price_data.loc[point, ['open', 'low', 'high', 'close']].mean()
#         elif price_type == 'close':
#             return self.price_data.loc[point,'close']
#         elif price_type=='open':
#             return self.price_data.iloc[point]['open']

#     '''
#     func to calculate PNL

#     '''
#     def calculate_pnl(self, action, previous_price, current_price, spread=True):

#         if  action==0:  # if action be 0(hold) pnl will be 0 
#             return 0

#         diff_price = current_price - previous_price

#         if spread:
#             return (action * diff_price - self.spread_cost * spread ) * self.num_of_shares
#         else:
#             return (action * diff_price ) * self.num_of_shares
             
#     '''
#     func to calculate reward

#     '''
#     def calculate_reward(self,current_position, new_action, current_point, previous_point):

#         previous_price = self.get_price(previous_point)
#         current_price = self.get_price(current_point)
#         next_price = self.get_price(current_point + 1)
#         profit_loss = self.calculate_pnl(current_position, previous_price, current_price)
#         profit_loss = (profit_loss if (new_action!=0) else 0)    # ignore profit_loss if new action be same as previous action
#         min_price = self.price_data['low'][previous_point:current_point].min()
#         max_price = self.price_data['high'][previous_point:current_point].max()
        
#         if current_position == 1:
#             MAE = self.calculate_pnl(current_position, previous_price, min_price, spread=False)
#             MPE = self.calculate_pnl(current_position, max_price, current_price, spread=False)
#         elif current_position == -1:
#             MAE = self.calculate_pnl(current_position, previous_price, max_price, spread=False)
#             MPE = self.calculate_pnl(current_position, min_price, current_price, spread=False)
#         else:
#             MAE = 0
#             MPE = 0

#         return profit_loss, MAE, MPE

#     '''
#     func to calculate account PNL in percent

#     '''
#     def calculate_benefit(self):

#         action = self.current_position
#         profit_loss = self.calculate_pnl(action, self.get_price(self.entry_index),
#                                          self.get_price(self.current_point))
#         acount_pnl_percentage = (self.base_account + profit_loss) / self.base_account

#         return (acount_pnl_percentage - 1), profit_loss

#     '''
#     func to get environment state

#     '''
#     def get_state(self):

#         s_ = self.data.iloc[self.current_point - self.timesteps:self.current_point , :]
#         st= s_.values.reshape(1,100,49)
       
#         return st

#     '''
#     func calculate sl tp based on swing

#     '''
#     def calculate_sltp(self, current_index, action, fill_price):
        
#         minimum_risk = self.sl_value
#         self.risk = self.sl_value
#         if self.sl_mode == 'fixed':  #fix mode
#             if action == 1:
#                 self.stop_loss = fill_price - action * self.risk
#                 self.take_profit = fill_price + action * self.risk 
#             else:
#                  self.take_profit= fill_price + action * self.risk
#                  self.stop_loss= fill_price - action * self.risk 

#         else:   #Swing mode
#             if action == 1:
#                 low_data = self.price_data['low'][:current_index + 1]
#                 swing_low_indices = argrelextrema(np.array(low_data), lambda x, y: x < y, order=self.scipy_neighborhood, )[0]
#                 try:
#                     self.stop_loss = low_data[next(x for x in swing_low_indices[::-1] if low_data[x] < fill_price)]
#                 except: 
#                     self.stop_loss = fill_price - minimum_risk
#                 self.risk = abs(self.stop_loss - fill_price)
#                 self.take_profit = fill_price + self.risk * self.R2R

#             else:
#                 high_data = self.price_data['high'][:current_index+1]
#                 swing_high_indices = argrelextrema(np.array(high_data), lambda x, y: x > y, order=self.scipy_neighborhood)[0]
#                 try:
#                     self.stop_loss =  high_data[next(x for x in swing_high_indices[::-1] if high_data[x] > fill_price)]
#                 except:
#                     self.stop_loss = fill_price + minimum_risk
#                 self.risk = abs(self.stop_loss - fill_price)
#                 self.take_profit = fill_price - self.risk * self.R2R

#     '''
#     func to step action on environment 

#     '''
#     def step(self, action):
        
#         doned_action = int(action) - 1    # action space should be in (-1,0,1)
#         reward=0
#         self.price_data.loc[self.current_point,'action']= doned_action
#         # last_action = (self.tradeslist)['action'][len(self.tradeslist)-1]
#         done = 0
#         close_price = self.get_price(self.current_point)

#         if (self.day_index < len(self.day_indices)-1 and self.current_point == self.day_indices[self.day_index+1] -1 ) or \
#                 self.current_point == len(self.data) - 2:
#             done = 1 

#         if  (doned_action!=0 and self.sltp_flag):

#             self.calculate_sltp(self.current_point,doned_action,close_price)
#             self.sltp_flag= False

#         profit_loss, MAE, MPE = self.calculate_reward(self.last_action,doned_action, self.current_point,self.entry_index)
  
#         if profit_loss and self.last_action!= doned_action:
            
#             if profit_loss > 0:
#                 self.positives += profit_loss
#             else:
#                 self.negatives += profit_loss
#             self.account += profit_loss

       
#         if (doned_action!=0 and self.last_action!= doned_action) :
          
#             #print('sl: ',self.stop_loss,'tp: ',self.take_profit,'action: ',self.last_action)
#             self.tradeslist.loc[len(self.tradeslist)] = [self.entry_date, self.get_price(self.entry_index),
#                                                         self.last_action, self.get_date(),
#                                                         self.get_price(self.current_point), profit_loss,self.account, MAE,
#                                                         MPE,'reverse',self.stop_loss,self.take_profit]
#             self.entry_index = self.current_point
#             self.entry_date = self.get_date()
#             self.calculate_sltp(self.current_point,doned_action,close_price)
#             self.last_action= doned_action
#             #self.sltp_flag= False
            
#         elif (self.last_action== 1  and (close_price>= self.take_profit or close_price<= self.stop_loss)) or (self.last_action== -1 and (close_price<= self.take_profit or close_price>= self.stop_loss)):
#                 #print('action:',doned_action,'close:',close_price,'sl:',self.stop_loss,'tp:',self.take_profit)
#                 self.tradeslist.loc[len(self.tradeslist)] = [self.entry_date, self.get_price(self.entry_index),
#                                                             self.last_action, self.get_date(),
#                                                             self.get_price(self.current_point), profit_loss,self.account, MAE,
#                                                             MPE,'sltp',self.stop_loss,self.take_profit]
#                 self.entry_index = self.current_point
#                 self.entry_date = self.get_date()
#                 self.last_action= doned_action
#                 self.sltp_flag= True
        
#         self.current_position = doned_action
        
#         if self.account <= 0:
#             done = 1
            
#         self.current_point += 1
       
#         s_ = self.get_state()
      
#         return s_, float(reward), done, {"account_status":self.account}

#     '''
#     func to reset environment

#     '''
#     def reset(self):

#         self.positives = 0
#         self.negatives = 0
#         self.day_index +=1
#         self.current_point = self.day_indices[self.day_index]
    
#         #self.make_plot() # plot result at the end of each day(episode)
#         self.episode += 1
       
#         self.current_position = 0
#         self.entry_index = self.current_point

#         return self.get_state()

#     '''
#     func to ?

#     '''
#     def _seed(self, seed=None):     # we want to start with same arameters each time
#         self.np_random, seed = seeding.np_random(seed)
#         return [seed]
        
#     '''
#     func to ?

#     '''
#     def get_sb_env(self):     
#         e = DummyVecEnv([lambda: self])
#         obs = e.reset()
#         return e, obs


#     '''
#     func to plot result

#     '''
#     def make_plot(self):
#         plt.plot(self.tradeslist['pnl'], 'g')   # plot PNL from trade list
#         plt.savefig('results/PNL.png')      # save plot as png
#         plt.close()
    
#     '''
#     func to save actions that has been taken on each step

#     '''
#     def trade_history(self):

#         self.price_data.to_csv('actiones_taked.csv')

#         return self.tradeslist


#     '''
#     func to get results of backtesting

#     '''
#     def show_results(self):

#         results= self.tradeslist.copy()
#         #results['cum_pnl'] = results['pnl'].cumsum()
#         initial_balance= self.base_account
       
#         cagr_dd = -20
#         pf = 0
#         pessimistic_return_on_margin = -10
#         if len(results):
#             pos = np.sum(results['pnl'][results['pnl'] > 0])
#             neg = np.sum(results['pnl'][results['pnl'] < 0])
#             long_num = len(results['action'][results['action'] == 1])
#             short_num = len(results['action'][results['action'] == -1])
#             cagr = float(results['cum_pnl'][-1:])
#             max_drawdown = np.min(
#                 results['cum_pnl'] - results['cum_pnl'].rolling(len(results), min_periods=1).max())
#             mean_drawdown = -np.mean(
#                 results['cum_pnl'] - results['cum_pnl'].rolling(len(results), min_periods=1).max())
#             cagr_dd = cagr / mean_drawdown if mean_drawdown != 0 else cagr
#             pf = pos / -neg if neg else np.sum(results['pnl'] > 0)
#             margin = 50
#             average_win = 0
#             average_loss = 0
#             winning_trades = len(results[results['pnl'] > 0].index)
#             losing_trades = len(results[results['pnl'] < 0].index)
#             if winning_trades:
#                 average_win = pos / winning_trades
#             if losing_trades:
#                 average_loss = - neg / losing_trades
#             adjusted_num_of_wins = winning_trades - winning_trades ** .5
#             adjusted_num_of_losses = losing_trades + losing_trades ** .5
#             adjusted_profit = average_win * adjusted_num_of_wins
#             adjusted_loss = average_loss * adjusted_num_of_losses
#             pessimistic_return_on_margin = (adjusted_profit - adjusted_loss) / margin
#             print(tabulate(
#                 {"long#":[long_num], "short#":[short_num], "Cumulative PnL": [cagr], "PROM": [pessimistic_return_on_margin],
#                 "Profit factor": [pf], "max drawdown": [max_drawdown], "PnLs": {str(pos) + ' ' + str(neg)}}, headers="keys", tablefmt="grid"))
                
#             plt.figure(figsize=(9,6))
#             plt.plot(results['exit date'],results['cum_pnl'] , color ='green', markersize=4)
#             plt.grid()
#             plt.ylabel('Point', labelpad = 8, fontsize = 14)
#             plt.xlabel('Dates', labelpad = 8, fontsize = 14)
#             plt.xticks(rotation=90)
#             plt.savefig('results/PNL.png') 
#             plt.show()
            